<a href="https://colab.research.google.com/github/vachkim/LECTURE/blob/master/MINING/Lec_MOVIE_REVIEW_DEEPLEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pip install --upgrade pip

pip install tensorflow

pip install tf-nightly


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/'My Drive'/'Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
%ls

'모두의 데이터 분석'/          기온공공데이터.ipynb
 0826_Class.ipynb              machinelearning_data/
 강의_BeautifulSoup.ipynb      Matplotlib_시각화.ipynb
 deeplearning_data/            naivebayes_example.csv
 강의_판다스.ipynb             ratings_test.txt
 강의_파이썬.ipynb             ratings_train.txt
 강의_넘파이.ipynb             SamsungCard_분석.ipynb
 강의_코딩실습.ipynb           SamsungCard.ipynb
 강의_영어마이닝.ipynb         samsung_data/
 강의_감정분석.ipynb           Seaborn_시각화.ipynb
 강의_한글마이닝.ipynb         강의_Selenium.ipynb
 강의_영화리뷰감정분석.ipynb   titanic_data/
 영화리뷰_딥러닝.ipynb         Titanic_기초분석.ipynb
 데이터시각화기초.ipynb        Untitled
 인구공공데이터.ipynb          weather_data/


In [ ]:
import pandas as pd
test = pd.read_csv('ratings_test.txt',sep='\t',quoting=3)
train = pd.read_csv('ratings_train.txt',sep='\t',quoting=3)

In [ ]:
train.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
train= train.dropna(how = 'any')

In [ ]:
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
import numpy as np
train['document'].replace('', np.nan, inplace=True)
print(train.isnull().sum())

id            0
document    391
label         0
dtype: int64


In [ ]:
train.loc[train.document.isnull()][:5]

,id,document,label
584,7117896,NaN,0
593,6478189,NaN,0
638,9364602,NaN,0
668,1600635,NaN,0
1559,6918082,NaN,1


In [ ]:
train = train.dropna(how = 'any')

In [ ]:
!pip install konlpy
from konlpy.tag import Okt

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
test.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test['document'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test = test.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test))

전처리 후 테스트용 샘플의 개수 : 48995


In [ ]:
okt = Okt()
X_train = []
for sentence in train['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
okt = Okt()
X_test = []
for sentence in test['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 43752
등장 빈도가 2번 이하인 희귀 단어의 수: 24337
단어 집합에서 희귀 단어의 비율: 55.62488571950996
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8715872104872904


In [ ]:
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 19417


In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
y_train = np.array(train['label'])
y_test = np.array(test['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

145380
145380


In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.0830925849498


In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1938/1939 [============================>.] - ETA: 0s - loss: 0.3888 - acc: 0.8238
Epoch 00001: val_acc improved from -inf to 0.84400, saving model to best_model.h5
1939/1939 [==============================] - 153s 79ms/step - loss: 0.3888 - acc: 0.8238 - val_loss: 0.3511 - val_acc: 0.8440
Epoch 2/15
1938/1939 [============================>.] - ETA: 0s - loss: 0.3273 - acc: 0.8577
Epoch 00002: val_acc improved from 0.84400 to 0.85528, saving model to best_model.h5
1939/1939 [==============================] - 154s 79ms/step - loss: 0.3273 - acc: 0.8577 - val_loss: 0.3330 - val_acc: 0.8553
Epoch 3/15
1938/1939 [============================>.] - ETA: 0s - loss: 0.3020 - acc: 0.8713
Epoch 00003: val_acc improved from 0.85528 to 0.85985, saving model to best_model.h5
1939/1939 [==============================] - 152s 78ms/step - loss: 0.3020 - acc: 0.8713 - val_loss: 0.3278 - val_acc: 0.8599
Epoch 4/15
1938/1939 [============================>.] - ETA: 0s - loss: 0.2840 - acc: 0.881

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1532/1532 [==============================] - 16s 11ms/step - loss: 0.3444 - acc: 0.8542

 테스트 정확도: 0.8542


In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))